In [62]:
import pandas as pd
import numpy as np
from math import log, floor, pow

class Analysis():
    def computeLogReturns(self, back, ahead):
        return np.divide(back, ahead).map(log)
    
    def generateGaussians(self, mean=100, std=1, n=1000):
        return np.random.normal(mean, std, n)
    
    def leadingDigit(self, x):
        absx = abs(x)
        return floor(float(absx) / pow(10, floor(log(absx))))
    
analysis = Analysis()

GSPCdata = pd.read_csv('./data/GSPC_historical_prices.csv')
GSPCvalues = GSPCdata['Adj Close']
times = data['Date']
timesOffByOne = times[1:]

logrets = analysis.computeLogReturns(GSPCvalues[:-1], GSPCvalues[1:])

In [61]:
popData = pd.read_csv('./data/PEP_2014_PEPANNRSIP.US12A_with_ann.csv')
citiesAndPopulation = popData[['GC_RANK.display-label.1', 'respop72014']]
